## Stitch the 110 Cal-CRAI metric maps into png files containing 6 grouped maps each
* input is a folder containing the 110 png metric map files
* consistent image sizing
* consistent spacing between plots
* consistent image cropping

In [ ]:
import os
from PIL import Image, ImageChops

In [ ]:
# Constants
INPUT_FOLDER = 'plots_directory'
OUTPUT_FOLDER = 'stitched_metrics'
IMAGES_PER_GROUP = 6
GRID_COLUMNS = 2
wh_ratio = 200/280 # width/height ratio
IMAGE_SIZE = (int(280*2*wh_ratio), 280*2)  # Resize images (width, height)
PADDING = 10  # Space between images

# Ensure the output folder exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Get sorted list of PNG files
image_files = sorted([f for f in os.listdir(INPUT_FOLDER) if f.endswith('.png')])

# Function to auto-crop whitespace from an image
def crop_whitespace(image):
    # Convert image to grayscale
    grayscale = image.convert("L")
    inverted = ImageChops.invert(grayscale)
    bbox = inverted.getbbox()
    if bbox:
        return image.crop(bbox)
    return image

# Function to create a grid image from a list of image paths
def create_grid(images, grid_columns, image_size, padding):
    grid_rows = (len(images) + grid_columns - 1) // grid_columns
    # Calculate grid dimensions, accounting for padding
    grid_width = grid_columns * (image_size[0] + padding) - padding
    grid_height = grid_rows * (image_size[1] + padding) - padding

    # Create a blank canvas for the grid
    grid_image = Image.new('RGB', (grid_width, grid_height), color=(255, 255, 255))

    for index, image_path in enumerate(images):
        img = Image.open(image_path)
        img = crop_whitespace(img)
        img = img.resize(image_size, Image.Resampling.LANCZOS)

        # Calculate position with padding
        x = (index % grid_columns) * (image_size[0] + padding)
        y = (index // grid_columns) * (image_size[1] + padding)
        grid_image.paste(img, (x, y))

    return grid_image

# Group images and create grid images
for i in range(0, len(image_files), IMAGES_PER_GROUP):
    group = image_files[i:i + IMAGES_PER_GROUP]
    image_paths = [os.path.join(INPUT_FOLDER, img) for img in group]

    # Create the grid image
    grid_image = create_grid(image_paths, GRID_COLUMNS, IMAGE_SIZE, PADDING)

    # Save the output
    output_path = os.path.join(OUTPUT_FOLDER, f'group_{i // IMAGES_PER_GROUP + 1}.png')
    grid_image.save(output_path, dpi=(500,500))
    print(f'Saved: {output_path}')